# DEM Analysis & Manipulation

Today's script is for doing some more advanced things with DEM data. We can extract more information than simply "elevation" from DEMs, and this can have some good scientific value. For example, we can extract such things as slope, aspect and curvature - these values are important for characterising many landforms such as drumlins. We can often also calculate more complex metrics, but at that point we approach the edge of GEE's capabilities, and move towards standard geospatial python tools.

In [10]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt

# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n0') # PUT YOUR API KEY (Project ID) HERE!

In [11]:
## Importing some elevation data
# First, ArcticDEM, this is an arctic region 2m mosaic, its surprisingly high-quality
# for such a large-scale dataset.

ArcticDEM = ee.Image("UMN/PGC/ArcticDEM/V4/2m_mosaic")

# Next, lets also import a collection of data
# Let's get the 1m data for France

FranceDEM = ee.ImageCollection("IGN/RGE_ALTI/1M/2_0")

In [12]:
## Produce a test plot of ArcticDEM to show coverage
Map = geemap.Map()
Map.addLayer(ArcticDEM, {'bands': 'elevation', 'min': -600, 'max': 4000}, "ArcticDEM")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

TraitError: Invalid selection: value not found

In [13]:
## Now we need to define some study areas
# For ArcticDEM first, seeing as there is coverage of Norway, let's focus there.

# Geirangerfjorden and Ålesund bounding box (approximate coordinates)
geiranger_bbox = ee.Geometry.Rectangle([6.0, 62.0, 7.5, 62.5])

# For France, lets pick somewhere with interesting relief.
# Cévennes National Park bounding box (approximate coordinates)
cevennes_bbox = ee.Geometry.Rectangle([3.00, 44.00, 4.00, 44.50])

# Now, as the French data is a data collection, we need to produce a mosaic based
# on the bounding box, lets do this below
cevennes_mosaic = FranceDEM.filterBounds(cevennes_bbox).mosaic()

# We should also clip both datasets to the bounding boxes
geiranger_mosaic = ArcticDEM.clip(geiranger_bbox)
cevennes_mosaic = cevennes_mosaic.clip(cevennes_bbox)

In [8]:
## Produce a test plot of ArcticDEM to show coverage
Map = geemap.Map()

# Check band names for geiranger_mosaic
print("Geiranger Mosaic Bands:", geiranger_mosaic.bandNames().getInfo())
# Check band names for cevennes_mosaic
print("Cévennes Mosaic Bands:", cevennes_mosaic.bandNames().getInfo())


# Update visualization parameters based on band names
geiranger_vis = {'bands': geiranger_mosaic.bandNames().getInfo()[0], 'min': -600, 'max': 4000}
cevennes_vis = {'bands': cevennes_mosaic.bandNames().getInfo()[0], 'min': -600, 'max': 4000}


Map.addLayer(geiranger_mosaic, geiranger_vis, "Geiranger")
Map.addLayer(cevennes_mosaic, cevennes_vis, "Cévennes")

Map

Geiranger Mosaic Bands: ['elevation', 'count', 'mad', 'mindate', 'maxdate', 'datamask']
Cévennes Mosaic Bands: ['MNT', 'SRC', 'DST']


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

TraitError: Invalid selection: value not found

## So, we have some DEM data - what can we do with it?

We can extract many parameters from it!
Let's work with the Geiranger data for now, as its a bit closer to home - if you want to work with the French data, go ahead of course!

Most importantly, we need a way to do some terrain analysis. There are a few native tools in GEE for this.

In [ ]:
## We are going to use the ee.Terrain tools
# First, lets try making a hillshade, this is simple enough
# We should set a factor of exaggeration for visualisation purposes
exag = 10
# Now we can produce the hillshade
geiranger_hillshade = ee.Terrain.hillshade(geiranger_mosaic.multiply(exag))


# Easy, right?

Let's try the other metrics that are available so far...

In [ ]:
## Aspect - this is the angle which a slope faces
Geiranger_aspect = ee.Terrain.aspect(geiranger_mosaic)

# We can also get slope, which is a valuable metric too
Geiranger_slope = ee.Terrain.slope(geiranger_mosaic)


## For more complexity, we need to make things a bit harder
Time to import a third-party module. This is tagee, it is a module which is a terrain analysis toolbox for google earth engine, it has some of the already existing functions we worked with now, but with some extra.
https://github.com/zecojls/tagee

In [ ]:
## First we need to install and call it
%pip install tagee
from tagee import terrainAnalysis

ModuleNotFoundError: No module named 'tagee'

In [ ]:
# Extract the 'elevation' band from geiranger_mosaic to create a single-band image
# This is needed as tagee doesn't work particularly well with multiband origin data
geirangerDEM = geiranger_mosaic.select('elevation')

In [ ]:
## Let's iterate tagee
# We do this with the following line:
geiranger_terrainattr = terrainAnalysis(geirangerDEM)

# Now we can produce and display the map data
# Tagee bands will be visible through the custom menu of the dataset, as will the data in other layers.
# At this point, you are able to determine specific information about each part of the DEM.
# Functions like tagee's terrain attributes can work better with a smoothed DEM.


GRMAP = geemap.Map()
GRMAP.addLayer(geiranger_mosaic, {}, 'DEM')
GRMAP.addLayer(geiranger_hillshade, {},  'Hillshade')
GRMAP.addLayer(geiranger_terrainattr, {}, 'Terrainattr')

GRMAP

In [ ]:
## Producing a smoothed DEM
#gaussianFilter = ee.Kernel.gaussian(
#  radius=3, sigma=2, units='pixels', normalize=True
#)
#srtmSmooth = ee.Image("DEM").convolve(gaussianFilter).resample("bilinear")

## srtmSmooth would go into this function in this case:
# srtm_terrainattr = terrainAnalysis(srtmSmooth)

# So, how do we analyse this data?

We can do this in many ways, first, we can sample pixels, beyond that, there are more complex approaches, such as the clustering method outlined in the GEETests script.

This script only goes this far, however below are some tutorials and documentation towards doing further terrain analysis. We may also go over some more in the next session. See here:



*   Extracting raster values for points: https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points
*   K-means clustering, an unsupervised machine learning approach (good for pattern recognition): https://developers.google.com/earth-engine/apidocs/ee-clusterer-wekakmeans
*   Random Forest, a supervised machine learning approach (good for finding things you can already define) https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest




